In [1]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from jax.config import config
config.update("jax_enable_x64", True)
from jax import numpy as jnp
import torch
from transformers import (
    BaseModelOutputWithPastAndCrossAttentions,
    BfBaseModelOutputWithPastAndCrossAttentions,
)
from utils import check_model_outputs_allclose
torch.manual_seed(0)
from brunoflow import Node

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
a, b, c = torch.randn(size=(2, 768)), torch.randn(size=(2, 768)), torch.randn(size=(2, 768))
last_hidden_state_torch = torch.randn(size=(2, 19, 768))
past_key_values_torch = ((a, b), (c,))
all_hidden_states_torch = None
attentions_torch = (c, b, a)
cross_attentions_torch = None

last_hidden_state_bf = Node(jnp.array(last_hidden_state_torch.numpy()))
past_key_values_bf = ((Node(jnp.array(a.numpy())), Node(jnp.array(b.numpy()))), (Node(jnp.array(c.numpy())),))
all_hidden_states_bf = all_hidden_states_torch
attentions_bf = (Node(jnp.array(c.numpy())), Node(jnp.array(b.numpy())), Node(jnp.array(a.numpy())))
cross_attentions_bf = cross_attentions_torch

In [3]:
output_torch = BaseModelOutputWithPastAndCrossAttentions(
    last_hidden_state=last_hidden_state_torch,
    past_key_values=past_key_values_torch,
    hidden_states=all_hidden_states_torch,
    attentions=attentions_torch,
    cross_attentions=cross_attentions_torch,
)

output_bf = BfBaseModelOutputWithPastAndCrossAttentions(
    last_hidden_state=last_hidden_state_bf,
    past_key_values=past_key_values_bf,
    hidden_states=all_hidden_states_bf,
    attentions=attentions_bf,
    cross_attentions=cross_attentions_bf,
)

check_model_outputs_allclose(output_bf, output_torch)

BfBaseModelOutputWithPastAndCrossAttentions BaseModelOutputWithPastAndCrossAttentions
Comparing equality of torch object <class 'NoneType'> with bf object <class 'NoneType'>.
Comparing equality of torch object <class 'NoneType'> with bf object <class 'NoneType'>.
